In [ ]:
!pip install pytorch_pretrained_bert 
!pip install datasets
!pip install transformers
!pip install rouge_score

In [ ]:
import json
from pytorch_pretrained_bert import cached_path
from tqdm import tqdm
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
import numpy as np
import nltk
import gc

In [ ]:
nltk.download('punkt')

In [ ]:
url = "https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json"

In [ ]:
personachat_file = cached_path(url)
with open(personachat_file, "r", encoding="utf-8") as f:
    dataset = json.loads(f.read())

In [ ]:
replicas_valid = []
sumarized_valid = []

for dialog in tqdm(dataset["valid"]):
    sumarized_valid += [" ".join(dialog["personality"])]
    replicas_valid += [" ".join(dialog["utterances"][-1]["history"][1::2])]

replicas_train = []
sumarized_train = []

for dialog in tqdm(dataset["train"]):
    sumarized_train += [" ".join(dialog["personality"])]
    replicas_train += [" ".join(dialog["utterances"][-1]["history"][1::2])]

In [ ]:
df_train = pd.DataFrame({"text": replicas_train, "summary": sumarized_train})
df_valid = pd.DataFrame({"text": replicas_valid, "summary": sumarized_valid})

In [ ]:
df_train = Dataset.from_pandas(df_train)
df_valid = Dataset.from_pandas(df_valid)

In [ ]:
ds = DatasetDict()

In [ ]:
ds['train'] = df_train
ds['test'] = df_valid

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)

    labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = ds.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
metric = load_metric("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    predict_with_generate=True,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=10,
    eval_steps = 250,
    evaluation_strategy="steps",
    fp16=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()